In [ ]:
%pylab nbagg
import h5py
import os
import matplotlib.pyplot as plt
import pandas as pd

import scipy 
import scipy.fftpack
import seaborn as sns

from scipy.fft import fft, ifft
from scipy import signal

from scipy import stats

In [ ]:
os.chdir("C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets")
mydir = "C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets"

In [ ]:
def load_h5(filename):
    with h5py.File(filename, "r") as f:
        df = f['data'][:]
    return df 

In [ ]:
#import time
filename = mydir + "/Time_health.h5"

with h5py.File(filename, "r") as f:
     t = f['time'][:]

In [ ]:
os.chdir("C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Wilson Cowan/DATA")
#import connectivity file to know which region corresponds to which index in the matrices
filename = "Connectivity.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    regions_list = list(f['region_labels'])

regions = np.array(regions_list)

indices = [*range(0,76 , 1)]

In [ ]:
#load in healthy brains 9u45
H_var = {}
for i in range(100): #(100):
    if i == 0:
        pass
    else:
        filename = mydir + "/Health_" + str(i) + ".h5"
        H_var['H_' + str(i)] = load_h5(filename)    

In [ ]:
#remove first 500ms
list_deleterows = [*range(0,46000,1)]

for i in H_var:
    H_var[i]= np.delete(H_var[i], list_deleterows, 0)

In [ ]:
#load in depressed brains  
D_var = {}
for i in range(100):
    if i == 0:
        pass
    else:
        filename = mydir + "/Depr_" + str(i) + ".h5"
        D_var['D_' + str(i)] = load_h5(filename)

In [ ]:
for i in D_var:
    D_var[i]= np.delete(D_var[i], list_deleterows, 0)

In [ ]:
#load in stimulated brain: HF 
HF_var = {}
for i in range(40):
    if i == 0:
        pass
    else:
        filename = mydir + "/HF_" + str(i) + ".h5"
        HF_var['HF_' + str(i)] = load_h5(filename)

In [ ]:
#load in stimulated brain: LF
LF_var = {}
for i in range(100):
    if i == 0:
        pass
    else:
        filename = mydir + "/LF_" + str(i) + ".h5"
        LF_var['LF_' + str(i)] = load_h5(filename)

In [ ]:
#remove first 500ms
list_deleterows = [*range(0,46000,1)]

for i in H_var:
    H_var[i]= np.delete(H_var[i], list_deleterows, 0)
for i in D_var:
    D_var[i]= np.delete(D_var[i], list_deleterows, 0)
for i in HF_var:
    HF_var[i]= np.delete(HF_var[i], list_deleterows, 0)
for i in LF_var:
    LF_var[i]= np.delete(LF_var[i], list_deleterows, 0)

In [ ]:
list_deleterows = [*range(0,46000,1)]
t = np.delete(t, list_deleterows, 0)

In [ ]:
#check dimensions
print(shape(t))
print(shape(H_var['H_1']))
print(shape(LF_var['LF_1']))
print(shape(D_var['D_1']))
print(shape(HF_var['HF_1']))

In [ ]:

prefrontal = [17,18, 19, 55, 56, 57]


prefr_left = [55, 56, 57],[17,18, 19]
temp_left = [38, 39, 68, 69, 71, 72]
par_left = [51, 53, 54]
occ_left = [73, 74]


#frontal – temporal  (Leuchter, Li)
# --> PFCcl, PFCdl, PFCdm, PFCm, PFCorb, FEF, PFCvl
prefrontal = [7, 17,18, 19, 20, 21, 23, 45, 55, 56, 57, 58, 59,61]
#temporal --> TCc, TCi, TCs, A1, A2
temporal = [0, 1, 30, 31, 33, 38, 39, 68, 69, 71 ]

#frontal – occipital (Leuchter)
# --> occipital: V1, V2
occipital = [35, 36, 73,74]

#frontal – parietal (Leuchter, Li)
# parietal --> PCi, PCm, PCs
parietal = [ 13, 15, 16, 51, 53, 54]

indices = [0, 1,7,  13, 15, 16, 17,18, 19, 20, 21, 23,30, 31, 33,35, 36,38, 39, 45, 51, 53, 54, 55, 56, 57, 58, 59,61, 68, 69, 71 , 73,74]
print(len(indices))


In [ ]:
count = 0
for i in indices:
    for b in indices:
        if i == b:
            pass
        if i < b:
            pass
        else:
            for x in range(40):
                count = count + 1
print(count) #16240 loops necessary 

In [ ]:
#important variables of dataset
sample_period = 0.0009765625 #seconds         CHECKEN OF DIT JUIST IS
sf = 1/sample_period #sampling frequency 
sample_rate = 1024

#sns.set(font_scale=1.2)
dt = 0.01
t = t #time, defined when loading in

In [ ]:
#LOOP FOR LOW ALPHA BAND: prefrontal- prefrontal ROIs 
c1 = np.repeat("", 1)
c2 = np.repeat("",1)
cRest = numpy.zeros(shape=(1,17))
sig_array = np.column_stack([c1, c2, cRest])
sig_array_tot = np.column_stack([c1, c2, cRest])

p_count = 0
paircounter = 0

indices = [0, 1,7,  13, 15, 16, 17,18, 19, 20, 21, 23,30, 31, 33,35, 36,38, 39, 45, 51, 53, 54, 55, 56, 57, 58, 59,61, 68, 69, 71 , 73,74]

for ROI1 in  indices:
    for ROI2 in  indices:
        if ROI1 == ROI2:
            pass
        elif ROI1 < ROI2:
            pass 
        
        else:
            count_h = 0
            count_d = 0
            count_hf = 0
            count_lf = 0
            for i in H_var:
                #healthy 
                xh = H_var[i][:, ROI1]
                yh = H_var[i][:, ROI2]
                cxy_h, f_h = cohere(xh, yh, 256, 1. / dt)
                cxy_low_h, f_low_h = cxy_h[21:26], f_h[21:26]
                
                if count_h == 0:
                    health_low = cxy_low_h
                else:
                    health_low = np.append(health_low, cxy_low_h)
                count_h = count_h + 1
                
            for i in D_var:
                #depressed 
                xd = D_var[i][:, ROI1]
                yd = D_var[i][:, ROI2]
                cxy_d, f_d = cohere(xd, yd, 256, 1. / dt)
                cxy_low_d, f_low_d = cxy_d[21:26], f_d[21:26]
                
                if count_d == 0:
                    dep_low = cxy_low_d
                else:
                    dep_low = np.append(dep_low, cxy_low_d)
                    
                count_d = count_d + 1
            
            for i in HF_var:
                #HF stimulation 
                xhf = HF_var[i][:, ROI1]
                yhf = HF_var[i][:, ROI2]
                cxy_hf, f_hf = cohere(xhf, yhf, 256, 1. / dt)
                cxy_low_hf, f_low_hf = cxy_hf[21:26], f_hf[21:26]
                
                if count_hf == 0:
                    hf_low = cxy_low_hf
                else:
                    hf_low = np.append(hf_low, cxy_low_hf)
                count_hf = count_hf + 1
            
            for i in LF_var:
                #LF stimulation 
                xlf = LF_var[i][:, ROI1]
                ylf = LF_var[i][:, ROI2]
                cxy_lf, f_lf = cohere(xlf, ylf, 256, 1. / dt)
                cxy_low_lf, f_low_lf = cxy_lf[21:26], f_lf[21:26]
                
                if count_lf == 0:
                    lf_low = cxy_low_lf
                else:
                    lf_low = np.append(lf_low, cxy_low_lf)
                    
                count_lf = count_lf + 1
                
            p_h_d = stats.ttest_ind(dep_low,health_low).pvalue #healthy vs depressed
            p_h_hf = stats.ttest_ind(hf_low,health_low).pvalue #healthy vs HF
            p_h_lf = stats.ttest_ind(lf_low,health_low).pvalue #healthy vs LF
            p_d_hf = stats.ttest_ind(dep_low,hf_low).pvalue #depr vs HF
            p_d_lf = stats.ttest_ind(dep_low,lf_low).pvalue #depr vs LF
            
            paircounter = paircounter + 1
            print(paircounter)
            print(p_h_d)
            if p_h_d <= 0.05 or p_h_hf <= 0.05 or p_h_lf <= 0.05 or p_d_hf <= 0.05 or p_d_lf <= 0.05:
                print("yes")
                sig_array[0][0] = str(regions[ROI1])
                sig_array[0][1] = str(regions[ROI2])
                sig_array[0][2] = mean(dep_low)
                sig_array[0][3] = mean(health_low)
                sig_array[0][4] = mean(hf_low)
                sig_array[0][5] = mean(lf_low)
                sig_array[0][6] = std(dep_low)
                sig_array[0][7] = std(health_low)
                sig_array[0][8] = std(hf_low)
                sig_array[0][9] = std(lf_low)
                sig_array[0][10] = p_h_d
                sig_array[0][11] = p_h_hf
                sig_array[0][12] = p_h_lf
                sig_array[0][13] = p_d_hf
                sig_array[0][14] = p_d_lf
                sig_array[0][15] = ROI1
                sig_array[0][16] = ROI2
                sig_array[0][17] = min(f_low_h)
                sig_array[0][18] = max(f_low_h)
                
                if p_count == 0:
                    sig_array_tot[0][0] = str(regions[ROI1])
                    sig_array_tot[0][1] = str(regions[ROI2])
                    sig_array_tot[0][2] = mean(dep_low)
                    sig_array_tot[0][3] = mean(health_low)
                    sig_array_tot[0][4] = mean(hf_low)
                    sig_array_tot[0][5] = mean(lf_low)
                    sig_array_tot[0][6] = std(dep_low)
                    sig_array_tot[0][7] = std(health_low)
                    sig_array_tot[0][8] = std(hf_low)
                    sig_array_tot[0][9] = std(lf_low)
                    sig_array_tot[0][10] = p_h_d
                    sig_array_tot[0][11] = p_h_hf
                    sig_array_tot[0][12] = p_h_lf
                    sig_array_tot[0][13] = p_d_hf
                    sig_array_tot[0][14] = p_d_lf
                    sig_array_tot[0][15] = ROI1
                    sig_array_tot[0][16] = ROI2
                    sig_array_tot[0][17] = min(f_low_h)
                    sig_array_tot[0][18] = max(f_low_h)
                    
                else:
                    sig_tuple = (sig_array_tot, sig_array)
                    sig_array_tot = np.vstack(sig_tuple)
                
                p_count = p_count + 1


file_name1 = mydir + "/coherence_low_selectedROIs.csv"
outcome1 = pd.DataFrame.from_records(sig_array_tot)
outcome1.columns = ["ROI1", "ROI2", "mean_coh_d", "mean_coh_h", "mean_coh_hf", "mean_coh_lf", "sd_coh_d", "sd_coh_h",
                    "sd_coh_hf","sd_coh_lf","p-H_D", "p-H-HF","p-H_LF","p-D_HF","p-D_LF" ,"ROI1_nr","ROI2_nr", "f_min", "f_max"]
outcome1.to_csv(path_or_buf = file_name1, index = False)

In [ ]:
#9u1